# dependencies
connect to cpu if want to use behaviour cloning

In [1]:
#!git clone http://github.com/HumanCompatibleAI/imitation
#!pip install -e "/content/imitation[dev]"

In [2]:
!pip install gymnasium[mujoco]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.9/953.9 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 33.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.8/211.8 kB 27.9 MB/s eta 0:00:00


In [3]:
!pip install imitation
!pip install shimmy
!pip install git+https://github.com/Stable-Baselines-Team/stable-baselines3-contrib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.4/216.4 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.7/181.7 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.9/107.9 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.4/413.4 kB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 19.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 12.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.4/195.4 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.1 MB/s eta 0:00:00
  Created wheel for docopt: filename=docopt-0.6.2-py2

# behaviour cloning

(imitation library, BC does not work as expected - RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu! (when checking argument for argument index in method wrapper_CUDA_gather)). Try run BC with cpu to avoid this issue (changing runtime type from GPU to CPU, and it can run on colab but having some issue on my Win10 laptop).

In [4]:
import gymnasium as gym
import imitation

In [5]:
import numpy as np
import gymnasium as gym
from stable_baselines3.common.evaluation import evaluate_policy

from imitation.algorithms import bc
from imitation.data import rollout
from imitation.data.wrappers import RolloutInfoWrapper
from imitation.policies.serialize import load_policy
from imitation.util.util import make_vec_env
from stable_baselines3 import PPO
from stable_baselines3.ppo import MlpPolicy
from stable_baselines3.common.evaluation import evaluate_policy

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [6]:
SEED = 42
rng = np.random.default_rng(0)
env = make_vec_env(
    "Reacher-v4", # seals:seals/CartPole-v0
    rng=np.random.default_rng(SEED),
    n_envs=1,
    post_wrappers=[
        lambda env, _: RolloutInfoWrapper(env)
    ],  # needed for computing rollouts later
)


In [7]:
model = PPO("MlpPolicy", env, learning_rate = 1e-4, batch_size=128, n_epochs = 10, verbose=1)
model.learn(total_timesteps=1000000)
model.save("ppo_Reacher-v4")

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


流式输出内容被截断，只能显示最后 5000 行内容。
|    loss                 | 0.0242       |
|    n_updates            | 2610         |
|    policy_gradient_loss | -0.00827     |
|    std                  | 0.0732       |
|    value_loss           | 0.0426       |
------------------------------------------
----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 50         |
|    ep_rew_mean          | -7.35      |
| time/                   |            |
|    fps                  | 575        |
|    iterations           | 263        |
|    time_elapsed         | 935        |
|    total_timesteps      | 538624     |
| train/                  |            |
|    approx_kl            | 0.01069546 |
|    clip_fraction        | 0.0826     |
|    clip_range           | 0.2        |
|    entropy_loss         | 2.4        |
|    explained_variance   | 0.996      |
|    learning_rate        | 0.0001     |
|    loss                 | 0.0316     |
|    n_updates    

In [10]:
expert = load_policy(
    "ppo",
    path = "/content/ppo_Reacher-v4.zip",
    venv=env,
)

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [11]:
expert_reward, _ = evaluate_policy(
    expert, env, 10, return_episode_rewards=True
)
print(
    "expert rewards after training:",
    np.mean(expert_reward),
    "+/-",
    np.std(expert_reward),
)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


expert rewards after training: -4.2783739 +/- 1.6138408850354766


# sep

In [ ]:
expert = load_policy(
    "ppo-huggingface",
    organization="HumanCompatibleAI",
    env_name="seals/Ant-v1",
    venv=env,
)

In [5]:
SEED = 42
rng = np.random.default_rng(0)
env = make_vec_env(
    "seals:seals/Ant-v1", # seals:seals/CartPole-v0
    rng=np.random.default_rng(SEED),
    n_envs=1,
    post_wrappers=[
        lambda env, _: RolloutInfoWrapper(env)
    ],  # needed for computing rollouts later
)
expert = load_policy(
    "ppo-huggingface",
    organization="HumanCompatibleAI",
    env_name="seals/Ant-v1",
    venv=env,
)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


ppo-seals-Ant-v1.zip:   0%|          | 0.00/187k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

In [ ]:
rollouts = rollout.rollout(
    expert,
    env,
    rollout.make_sample_until(min_timesteps=1000, min_episodes=1), #这里episode数量回合n_envs对齐，最少也会有n_envs条轨迹，受前面en影响
    rng=rng,
)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
rollout.rollout_stats(rollouts)

{'n_traj': 1,
 'return_min': 2613.5690003122854,
 'return_mean': 2613.5690003122854,
 'return_std': 0.0,
 'return_max': 2613.5690003122854,
 'len_min': 1000,
 'len_mean': 1000.0,
 'len_std': 0.0,
 'len_max': 1000}

In [ ]:
def linear_schedule(initial_value: float):
    """
    Linear learning rate schedule.

    :param initial_value: Initial learning rate.
    :return: schedule that computes
      current learning rate depending on remaining progress
    """
    def func(progress_remaining: float) -> float:
        """
        Progress will decrease from 1 (beginning) to 0.

        :param progress_remaining:
        :return: current learning rate
        """
        return progress_remaining * initial_value

    return func

In [ ]:
from stable_baselines3.ppo import MlpPolicy

transitions = rollout.flatten_trajectories(rollouts)

bc_trainer = bc.BC(
    observation_space=env.observation_space,
    policy = MlpPolicy(env.observation_space, env.action_space, linear_schedule(0.001)), # default FeedForward32Policy, 2 hidden layers of 32 units
    batch_size=512,
    action_space=env.action_space,
    demonstrations=transitions,
    rng=rng,
)

In [ ]:
reward, _ = evaluate_policy(bc_trainer.policy, env, 10, return_episode_rewards=True)

print(
    "Rewards before training:",
    np.mean(reward),
    "+/-",
    np.std(reward),
)

Rewards before training: 1000.4576907000001 +/- 5.388815693692241


In [ ]:
bc_trainer.train(n_epochs=1000)

0batch [00:00, ?batch/s]

--------------------------------
| batch_size        | 512      |
| bc/               |          |
|    batch          | 0        |
|    ent_loss       | -0.00492 |
|    entropy        | 4.92     |
|    epoch          | 0        |
|    l2_loss        | 0        |
|    l2_norm        | 263      |
|    loss           | 1.09     |
|    neglogp        | 1.09     |
|    prob_true_act  | 0.337    |
|    samples_so_far | 512      |
--------------------------------



Epoch 0 of 1000         
Epoch 1 of 1000
Epoch 2 of 1000
5batch [00:00, 42.19batch/s]
Epoch 4 of 1000             
Epoch 5 of 1000
Epoch 6 of 1000
Epoch 7 of 1000
10batch [00:00, 39.64batch/s]
Epoch 9 of 1000              
Epoch 10 of 1000
Epoch 11 of 1000
14batch [00:00, 37.75batch/s]
Epoch 13 of 1000             
Epoch 14 of 1000
Epoch 15 of 1000
Epoch 16 of 1000
19batch [00:00, 38.61batch/s]
Epoch 18 of 1000             
Epoch 19 of 1000
Epoch 20 of 1000
Epoch 21 of 1000
24batch [00:00, 39.49batch/s]
Epoch 23 of 1000             
Epoch 24 of 1000
Epoch 25 of 1000
28batch [00:00, 39.43batch/s]
Epoch 27 of 1000             
Epoch 28 of 1000
Epoch 29 of 1000
Epoch 30 of 1000
33batch [00:00, 40.22batch/s]
Epoch 32 of 1000             
Epoch 33 of 1000
Epoch 34 of 1000
Epoch 35 of 1000
38batch [00:00, 38.73batch/s]
Epoch 37 of 1000             
Epoch 38 of 1000
Epoch 39 of 1000
42batch [00:01, 38.65batch/s]
Epoch 41 of 1000             
Epoch 42 of 1000
Epoch 43 of 1000
46batch [00:01, 

---------------------------------
| batch_size        | 512       |
| bc/               |           |
|    batch          | 500       |
|    ent_loss       | -0.000989 |
|    entropy        | 0.989     |
|    epoch          | 500       |
|    l2_loss        | 0         |
|    l2_norm        | 310       |
|    loss           | -2.69     |
|    neglogp        | -2.69     |
|    prob_true_act  | 15        |
|    samples_so_far | 256512    |
---------------------------------



Epoch 500 of 1000
Epoch 501 of 1000
504batch [00:14, 36.74batch/s]
Epoch 503 of 1000             
Epoch 504 of 1000
Epoch 505 of 1000
508batch [00:14, 35.52batch/s]
Epoch 507 of 1000             
Epoch 508 of 1000
Epoch 509 of 1000
512batch [00:14, 36.58batch/s]
Epoch 511 of 1000             
Epoch 512 of 1000
Epoch 513 of 1000
516batch [00:14, 37.28batch/s]
Epoch 515 of 1000             
Epoch 516 of 1000
Epoch 517 of 1000
520batch [00:14, 37.96batch/s]
Epoch 519 of 1000             
Epoch 520 of 1000
Epoch 521 of 1000
524batch [00:14, 38.16batch/s]
Epoch 523 of 1000             
Epoch 524 of 1000
Epoch 525 of 1000
528batch [00:15, 38.69batch/s]
Epoch 527 of 1000             
Epoch 528 of 1000
Epoch 529 of 1000
532batch [00:15, 38.35batch/s]
Epoch 531 of 1000             
Epoch 532 of 1000
Epoch 533 of 1000
536batch [00:15, 38.66batch/s]
Epoch 535 of 1000             
Epoch 536 of 1000
Epoch 537 of 1000
540batch [00:15, 37.59batch/s]
Epoch 539 of 1000             
Epoch 540 of 1000
E

In [ ]:
reward, _ = evaluate_policy(bc_trainer.policy, env, 10, return_episode_rewards=True)

print(
    "Rewards after training:",
    np.mean(reward),
    "+/-",
    np.std(reward),
)

Rewards after training: 901.9277635000001 +/- 659.1629902160806


In [ ]:
expert_reward, _ = evaluate_policy(expert, env, 10, return_episode_rewards=True)
print(
    "expert reward:",
    np.mean(expert_reward),
    "+/-",
    np.std(expert_reward),
)

expert reward: 2644.9786512 +/- 283.70998037654283
